# Controller Debug
This code allows us to figure out any oddities in construction of the controller, including maximums and centering.

In [ ]:
import evdev, time
import pathlib 

for device in pathlib.Path('/dev/input/by-id/').glob('*Xbox*event-joystick'):
    print("Found joystick device:", device)
joy = evdev.InputDevice(device)
print(joy)

In [ ]:
print(f"Left Stick X:  {joy.absinfo(evdev.ecodes.ABS_X)}")
print(f"Left Stick Y:  {joy.absinfo(evdev.ecodes.ABS_Y)}")
print(f"Right Stick X: {joy.absinfo(evdev.ecodes.ABS_RX)}")
print(f"Right Stick Y: {joy.absinfo(evdev.ecodes.ABS_RY)}")

In [ ]:
now = time.time()
file = open('controller_debug_data.csv', 'wt', newline='\n')
while True:
    file.write(f"{time.time()-now},{joy.absinfo(evdev.ecodes.ABS_X).value},{joy.absinfo(evdev.ecodes.ABS_Y).value},{joy.absinfo(evdev.ecodes.ABS_RX).value},{joy.absinfo(evdev.ecodes.ABS_RY).value}\n")
    time.sleep(0.005)

In [ ]:
from ipycanvas import Canvas, hold_canvas
from collections import namedtuple 

Point = namedtuple("Point", "x y")

stick_x = evdev.ecodes.ABS_X
stick_y = evdev.ecodes.ABS_Y 

canvas_size = Point(500, 500)
canvas_gutter = Point(50, 50)
canvas_origin = Point(canvas_size.x//2 + canvas_gutter.x, canvas_size.y//2 + canvas_gutter.x)
joy_size = Point( joy.absinfo(stick_x).max - joy.absinfo(stick_x).min, 
                 joy.absinfo(stick_y).max - joy.absinfo(stick_y).min,
                 )
joy_origin = Point(0, 0)

canvas = Canvas(width=canvas_size.x+2*canvas_gutter.x, height=canvas_size.y+2*canvas_gutter.x)
display(canvas)

history_size = 100 
history = [Point(0,0)] * history_size
index = 0
while True:
    val = Point(
        (joy.absinfo(stick_x).value - joy_origin.x) * (canvas_size.x / joy_size.x) + canvas_origin.x,
        (joy.absinfo(stick_y).value - joy_origin.y) * (canvas_size.y / joy_size.y) + canvas_origin.y,
    )
    history[index] = val 
    index = (index + 1) % history_size 
    with hold_canvas():
        canvas.clear()
        for i, h in enumerate(history[index:] + history[0:index]):
            canvas.global_alpha = (i / history_size)
            canvas.fill_circle(h.x, h.y, 10)
        time.sleep(0.02)